In [ ]:
from gis.config import Config

config = Config()

import os 


labels = [x for x in os.listdir(config.mnt_path / 'label/18')]
labels

labelled_coords = []
for label in labels:
    x,y = label.split('_')
    x, y = int(x), int(y.replace('.npy', ''))
    labelled_coords.append((18, x, y))
len(labelled_coords)

In [ ]:
import cv2 
import matplotlib.pyplot as plt 
import numpy as np

def load_mask(coord):
    z,x,y = coord
    filepath = config.mnt_path / f'label/18/{x}_{y}.npy'
    mask = np.load(filepath)
    return mask


to_expand = [] 
for coord in labelled_coords:
    mask = load_mask(coord)
    has_track = np.mean(mask) > 0
    if has_track:
        to_expand.append(coord)
len(to_expand)

In [ ]:
to_expand[0]

In [ ]:
from gis.image_utils import load_mask
from gis.line import skeleton_to_wkt_multilinestring
from gis.tile import extend_line_and_get_tiles
from skimage import morphology
from scipy import ndimage
import matplotlib.pyplot as plt 
from shapely import wkt
import shapely
import random 

def get_new_tiles_for_track_direction(x,y, extend_dist=500):
    coord = (x,y)
    # First get the line going through mask + 2 lat/lon points on said line 
    mask = load_mask((18, coord[0], coord[1]))
    skeleton = morphology.skeletonize(mask)
    ls = skeleton_to_wkt_multilinestring(skeleton, coord[0],coord[1], 18)
    result = wkt.loads(ls)
    all_coords = []
    for line_string in result.geoms:
        coords_for_line = list(line_string.coords)
        all_coords.extend(coords_for_line)

    sample = random.sample(all_coords, 2)
    coord1 = sample[0]
    coord2 = sample[1]

    lat1, lon1 = coord1[1], coord1[0]
    lat2, lon2 = coord2[1], coord2[0]

    # Get new tiles by extending this guy 
    new_tiles = extend_line_and_get_tiles(lat1, lon1, lat2, lon2, extension_distance_m=extend_dist, zoom_level=18)
    return [(t.z, t.x, t.y) for t in new_tiles]

all_new_tiles = []
for track_tile in to_expand:
    t_ = get_new_tiles_for_track_direction(track_tile[1],track_tile[2])
    all_new_tiles.extend(t_)

all_new_tiles = list(set(all_new_tiles))
len(all_new_tiles)

In [ ]:
# import os 
# from gis.tile import download_tile

# added = 0
# for coord in all_new_tiles:
#     z, x, y = coord
#     filepath = config.mnt_path / f'image/18/{z}_{x}_{y}.jpg'
#     download_tile(z, x, y)


In [ ]:
# EXPANSION_FACTOR = 1
# new_coords = []
# for (z,x,y) in to_expand:
    
#     for i in range(-EXPANSION_FACTOR, EXPANSION_FACTOR+1, 1):
#         for j in range(-EXPANSION_FACTOR, EXPANSION_FACTOR+1, 1):
#             new_coords.append((z, x+i, y+j))
# new_coords = list(set(new_coords))
# len(new_coords)